## **CNN Modeling**

In [24]:
import numpy as np
import pandas as pd

# 벡터화된 데이터 불러오기
data = pd.read_json("./labeled_data.json")
data.columns = ["label", "trigram"]

In [25]:
# flastten: 3 x 50 -> 150
data['trigram'] = data['trigram'].apply(lambda x: (np.array(x).reshape(-1)))

In [ ]:
# train/test 데이터 분리
from sklearn.model_selection import train_test_split

y = data.pop('label')
X = data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)